In [ ]:
python
import pyaudio
import numpy as np
import webrtcvad
import opuslib
from pyogg import OggStreamBufferedEncoder, OggOpusWriter

# Constants
RATE = 16000
CHANNELS = 1
CHUNK = 480
FORMAT = pyaudio.paInt16

# Initialize
audio = pyaudio.PyAudio()
vad = webrtcvad.Vad(3)
encoder = opuslib.Encoder(RATE, CHANNELS, opuslib.APPLICATION_VOIP)
decoder = opuslib.Decoder(RATE, CHANNELS)
buffered_encoder = OggStreamBufferedEncoder(OggOpusWriter)

# Callback function for playing audio
def play_callback(in_data, frame_count, time_info, status):
    pcm_data = decoder.decode(in_data)
    return (pcm_data, pyaudio.paContinue)

# Callback function for recording audio
def record_callback(in_data, frame_count, time_info, status):
    audio_data = np.frombuffer(in_data, dtype=np.int16)
    is_speech = vad.is_speech(audio_data.tobytes(), RATE)

    if is_speech:
        encoded_data = encoder.encode(audio_data.tobytes(), CHUNK)
        buffered_encoder.add_packet(encoded_data)

        if buffered_encoder.is_buffer_ready():
            ogg_data = buffered_encoder.get_buffer()
            stream_out.write(ogg_data)

    return (in_data, pyaudio.paContinue)

# Open input and output streams
stream_in = audio.open(format=FORMAT, channels=CHANNELS, rate=RATE, input=True, frames_per_buffer=CHUNK, stream_callback=record_callback)
stream_out = audio.open(format=FORMAT, channels=CHANNELS, rate=RATE, output=True, frames_per_buffer=CHUNK, stream_callback=play_callback)

# Start the streams
stream_in.start_stream()
stream_out.start_stream()

# Keep the script running
try:
    while stream_in.is_active() and stream_out.is_active():
        time.sleep(0.1)
except KeyboardInterrupt:
    pass

# Stop the streams and clean up
stream_in.stop_stream()
stream_out.stop_stream()
stream_in.close()
stream_out.close()
audio.terminate()


In [1]:
import pyaudio
import numpy as np
import opuslib
from pyogg import OggStreamBufferedEncoder, OggOpusWriter
from pydub import AudioSegment
from pydub.silence import detect_nonsilent

Exception: Could not find Opus library. Make sure it is installed.

In [ ]:
python
import pyaudio
import numpy as np
import opuslib
from pyogg import OggStreamBufferedEncoder, OggOpusWriter
from pydub import AudioSegment
from pydub.silence import detect_nonsilent

# Constants
RATE = 16000
CHANNELS = 1
CHUNK = 480
FORMAT = pyaudio.paInt16
SILENCE_THRESHOLD = -50
REQUIRED_NONSILENT = 100

# Initialize
audio = pyaudio.PyAudio()
encoder = opuslib.Encoder(RATE, CHANNELS, opuslib.APPLICATION_VOIP)
decoder = opuslib.Decoder(RATE, CHANNELS)
buffered_encoder = OggStreamBufferedEncoder(OggOpusWriter)

# Detect if there's voice activity
def is_speech(audio_data):
    audio_segment = AudioSegment(audio_data, sample_width=2, channels=CHANNELS, frame_rate=RATE)
    nonsilent_ranges = detect_nonsilent(audio_segment, min_silence_len=100, silence_thresh=SILENCE_THRESHOLD)

    if nonsilent_ranges and nonsilent_ranges[-1][1] - nonsilent_ranges[0][0] >= REQUIRED_NONSILENT:
        return True
    return False

# Callback function for playing audio
def play_callback(in_data, frame_count, time_info, status):
    pcm_data = decoder.decode(in_data)
    return (pcm_data, pyaudio.paContinue)

# Callback function for recording audio
def record_callback(in_data, frame_count, time_info, status):
    if is_speech(in_data):
        audio_data = np.frombuffer(in_data, dtype=np.int16)
        encoded_data = encoder.encode(audio_data.tobytes(), CHUNK)
        buffered_encoder.add_packet(encoded_data)

        if buffered_encoder.is_buffer_ready():
            ogg_data = buffered_encoder.get_buffer()
            stream_out.write(ogg_data)

    return (in_data, pyaudio.paContinue)

# Open input and output streams
stream_in = audio.open(format=FORMAT, channels=CHANNELS, rate=RATE, input=True, frames_per_buffer=CHUNK, stream_callback=record_callback)
stream_out = audio.open(format=FORMAT, channels=CHANNELS, rate=RATE, output=True, frames_per_buffer=CHUNK, stream_callback=play_callback)

# Start the streams
stream_in.start_stream()
stream_out.start_stream()

# Keep the script running
try:
    while stream_in.is_active() and stream_out.is_active():
        time.sleep(0.1)
except KeyboardInterrupt:
    pass

# Stop the streams and clean up
stream_in.stop_stream()
stream_out.stop_stream()
stream_in.close()
stream_out.close()
audio.terminate()


In [2]:
import pyaudio
import numpy as np
from pydub import AudioSegment
from pydub.silence import detect_nonsilent


In [5]:

import pyaudio
import numpy as np
from pydub import AudioSegment
from pydub.silence import detect_nonsilent
import time
# Constants
RATE = 16000
CHANNELS = 1
CHUNK = 480
FORMAT = pyaudio.paInt16
SILENCE_THRESHOLD = -50
REQUIRED_NONSILENT = 100

# Initialize
audio = pyaudio.PyAudio()

# Detect if there's voice activity
def is_speech(audio_data):
    audio_segment = AudioSegment(audio_data, sample_width=2, channels=CHANNELS, frame_rate=RATE)
    nonsilent_ranges = detect_nonsilent(audio_segment, min_silence_len=100, silence_thresh=SILENCE_THRESHOLD)

    if nonsilent_ranges and nonsilent_ranges[-1][1] - nonsilent_ranges[0][0] >= REQUIRED_NONSILENT:
        return True
    return False

# Callback function for playing audio
def play_callback(in_data, frame_count, time_info, status):
    return (in_data, pyaudio.paContinue)

# Callback function for recording audio
def record_callback(in_data, frame_count, time_info, status):
    if is_speech(in_data):
        stream_out.write(in_data)

    return (in_data, pyaudio.paContinue)

# Open input and output streams
stream_in = audio.open(format=FORMAT, channels=CHANNELS, rate=RATE, input=True, frames_per_buffer=CHUNK, stream_callback=record_callback)
stream_out = audio.open(format=FORMAT, channels=CHANNELS, rate=RATE, output=True, frames_per_buffer=CHUNK, stream_callback=play_callback)

# Start the streams
stream_in.start_stream()
stream_out.start_stream()

# Keep the script running
try:
    while stream_in.is_active() and stream_out.is_active():
        time.sleep(0.1)
except KeyboardInterrupt:
    pass

# Stop the streams and clean up
stream_in.stop_stream()
stream_out.stop_stream()
stream_in.close()
stream_out.close()
audio.terminate()


In [7]:
import pyaudio
import numpy as np
from collections import deque
from pydub import AudioSegment
from pydub.silence import detect_nonsilent

# Constants
RATE = 16000
CHANNELS = 1
CHUNK = 480
FORMAT = pyaudio.paInt16
SILENCE_THRESHOLD = -50
REQUIRED_NONSILENT = 100
BUFFER_SIZE = 10

# Initialize
audio = pyaudio.PyAudio()
ring_buffer = deque(maxlen=BUFFER_SIZE)

# Detect if there's voice activity
def is_speech(audio_data):
    audio_segment = AudioSegment(audio_data, sample_width=2, channels=CHANNELS, frame_rate=RATE)
    nonsilent_ranges = detect_nonsilent(audio_segment, min_silence_len=100, silence_thresh=SILENCE_THRESHOLD)

    if nonsilent_ranges and nonsilent_ranges[-1][1] - nonsilent_ranges[0][0] >= REQUIRED_NONSILENT:
        return True
    return False

# Callback function for playing audio
def play_callback(in_data, frame_count, time_info, status):
    if ring_buffer:
        return (ring_buffer.popleft(), pyaudio.paContinue)
    else:
        return (bytes(CHUNK * 2), pyaudio.paContinue)

# Callback function for recording audio
def record_callback(in_data, frame_count, time_info, status):
    voice_detected = is_speech(in_data)
    if voice_detected:
        print("Voice detected")
        ring_buffer.append(in_data)
    else:
        print("No voice detected")

    return (in_data, pyaudio.paContinue)

# Open input and output streams
stream_in = audio.open(format=FORMAT, channels=CHANNELS, rate=RATE, input=True, frames_per_buffer=CHUNK, stream_callback=record_callback)
stream_out = audio.open(format=FORMAT, channels=CHANNELS, rate=RATE, output=True, frames_per_buffer=CHUNK, stream_callback=play_callback)

# Start the streams
stream_in.start_stream()
stream_out.start_stream()

# Keep the script running
try:
    while stream_in.is_active() and stream_out.is_active():
        time.sleep(0.1)
except KeyboardInterrupt:
    pass

# Stop the streams and clean up
stream_in.stop_stream()
stream_out.stop_stream()
stream_in.close()
stream_out.close()
audio.terminate()


No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice d

In [8]:
import pyaudio

p = pyaudio.PyAudio()

for i in range(p.get_device_count()):
   dev = p.get_device_info_by_index(i)
   print(f"Device {i}: {dev['name']}")

Device 0: Microsoft Sound Mapper - Input
Device 1: Headset Microphone (Jabra EVOLV
Device 2: Microphone Array (IntelÂ® Smart 
Device 3: Microsoft Sound Mapper - Output
Device 4: Headset Earphone (Jabra EVOLVE 
Device 5: Speakers (Realtek(R) Audio)
Device 6: Primary Sound Capture Driver
Device 7: Headset Microphone (Jabra EVOLVE 20 MS)
Device 8: Microphone Array (IntelÂ® Smart Sound Technology for Digital Microphones)
Device 9: Primary Sound Driver
Device 10: Headset Earphone (Jabra EVOLVE 20 MS)
Device 11: Speakers (Realtek(R) Audio)
Device 12: Headset Earphone (Jabra EVOLVE 20 MS)
Device 13: Speakers (Realtek(R) Audio)
Device 14: Microphone Array (IntelÂ® Smart Sound Technology for Digital Microphones)
Device 15: Headset Microphone (Jabra EVOLVE 20 MS)
Device 16: Headphones (Realtek HD Audio 2nd output)
Device 17: Stereo Mix (Realtek HD Audio Stereo input)
Device 18: Speakers (Realtek HD Audio output with SST)
Device 19: Microphone (Realtek HD Audio Mic input)
Device 20: Microphone Ar

In [9]:
stream = p.open(format=pyaudio.paInt16,
               channels=1,
               rate=44100,
               input=True,
               input_device_index=1,
               frames_per_buffer=1024)

In [10]:
import webrtcvad

vad = webrtcvad.Vad()
vad.set_mode(3)  # Set aggressiveness, 3 is the highest

# Process audio frames with VAD
is_speech = vad.is_speech(audio_frame, sample_rate)

NameError: name 'audio_frame' is not defined

In [11]:

import pyaudio
import numpy as np
from collections import deque
from pydub import AudioSegment
from pydub.silence import detect_nonsilent
import time

# Constants
RATE = 16000
CHANNELS = 1
CHUNK = 480
FORMAT = pyaudio.paInt16
SILENCE_THRESHOLD = -50
REQUIRED_NONSILENT = 100
BUFFER_SIZE = 10

# Initialize
audio = pyaudio.PyAudio()
ring_buffer = deque(maxlen=BUFFER_SIZE)

# Detect if there's voice activity
def is_speech(audio_data):
    audio_segment = AudioSegment(audio_data, sample_width=2, channels=CHANNELS, frame_rate=RATE)
    nonsilent_ranges = detect_nonsilent(audio_segment, min_silence_len=100, silence_thresh=SILENCE_THRESHOLD)

    if nonsilent_ranges and nonsilent_ranges[-1][1] - nonsilent_ranges[0][0] >= REQUIRED_NONSILENT:
        return True
    return False

# Callback function for playing audio
def play_callback(in_data, frame_count, time_info, status):
    if ring_buffer:
        return (ring_buffer.popleft(), pyaudio.paContinue)
    else:
        return (bytes(CHUNK * 2), pyaudio.paContinue)

# Callback function for recording audio
def record_callback(in_data, frame_count, time_info, status):
    voice_detected = is_speech(in_data)
    if voice_detected:
        print("Voice detected")
        ring_buffer.append(in_data)
    else:
        print("No voice detected")

    return (in_data, pyaudio.paContinue)

# Open input and output streams
stream_in = audio.open(format=FORMAT, channels=CHANNELS, rate=RATE, input=True, input_device_index=1, frames_per_buffer=CHUNK, stream_callback=record_callback)
stream_out = audio.open(format=FORMAT, channels=CHANNELS, rate=RATE, output=True, frames_per_buffer=CHUNK, stream_callback=play_callback)

# Start the streams
stream_in.start_stream()
stream_out.start_stream()

# Keep the script running
try:
    while stream_in.is_active() and stream_out.is_active():
        time.sleep(0.1)
except KeyboardInterrupt:
    pass

# Stop the streams and clean up
stream_in.stop_stream()
stream_out.stop_stream()
stream_in.close()
stream_out.close()
audio.terminate()


No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice detected
No voice d

In [12]:
import pyaudio
import numpy as np
from collections import deque
from pydub import AudioSegment
from pydub.silence import detect_nonsilent

# Constants
RATE = 16000
CHANNELS = 1
CHUNK = 480
FORMAT = pyaudio.paInt16
SILENCE_THRESHOLD = -50
REQUIRED_NONSILENT = 100
BUFFER_SIZE = 10

# Initialize
audio = pyaudio.PyAudio()
ring_buffer = deque(maxlen=BUFFER_SIZE)

# Detect if there's voice activity
def is_speech(audio_data):
    audio_segment = AudioSegment(audio_data, sample_width=2, channels=CHANNELS, frame_rate=RATE)
    nonsilent_ranges = detect_nonsilent(audio_segment, min_silence_len=100, silence_thresh=SILENCE_THRESHOLD)

    if nonsilent_ranges and nonsilent_ranges[-1][1] - nonsilent_ranges[0][0] >= REQUIRED_NONSILENT:
        return True
    return False

# Callback function for playing audio
def play_callback(in_data, frame_count, time_info, status):
    if ring_buffer:
        return (ring_buffer.popleft(), pyaudio.paContinue)
    else:
        return (bytes(CHUNK * 2), pyaudio.paContinue)

# Callback function for recording audio
def record_callback(in_data, frame_count, time_info, status):
    if is_speech(in_data):
        ring_buffer.append(in_data)

    return (in_data, pyaudio.paContinue)

# Open input and output streams
stream_in = audio.open(format=FORMAT, channels=CHANNELS, rate=RATE, input=True, frames_per_buffer=CHUNK, stream_callback=record_callback)
stream_out = audio.open(format=FORMAT, channels=CHANNELS, rate=RATE, output=True, frames_per_buffer=CHUNK, stream_callback=play_callback)

# Start the streams
stream_in.start_stream()
stream_out.start_stream()

# Keep the script running
try:
    while stream_in.is_active() and stream_out.is_active():
        time.sleep(0.1)
except KeyboardInterrupt:
    pass

# Stop the streams and clean up
stream_in.stop_stream()
stream_out.stop_stream()
stream_in.close()
stream_out.close()
audio.terminate()


In [13]:

import sounddevice as sd
import soundfile as sf
import io
from pydub import AudioSegment
from pydub.silence import detect_nonsilent
import time
import numpy as np

# Constants
RATE = 16000
CHANNELS = 1
CHUNK = 480
SILENCE_THRESHOLD = -50
REQUIRED_NONSILENT = 100
RECORD_SECONDS = 5

# Detect if there's voice activity
def is_speech(audio_data):
    audio_segment = AudioSegment(audio_data, sample_width=2, channels=CHANNELS, frame_rate=RATE)
    nonsilent_ranges = detect_nonsilent(audio_segment, min_silence_len=100, silence_thresh=SILENCE_THRESHOLD)

    if nonsilent_ranges and nonsilent_ranges[-1][1] - nonsilent_ranges[0][0] >= REQUIRED_NONSILENT:
        return True
    return False

# Record audio using sounddevice
def record_audio():
    print("Recording...")
    audio_data = sd.rec(frames=RECORD_SECONDS * RATE, samplerate=RATE, channels=CHANNELS, dtype='int16')
    sd.wait()
    return audio_data

# Save recorded audio to a temporary WAV file and read it as a byte stream
def save_to_byte_stream(audio_data):
    with io.BytesIO() as byte_stream:
        with sf.SoundFile(byte_stream, 'w', samplerate=RATE, channels=CHANNELS, format='WAV', subtype='PCM_16') as wav_file:
            wav_file.write(audio_data)
        byte_stream.seek(0)
        audio_segment = AudioSegment.from_file(byte_stream, format="wav")
    return audio_segment

# Convert audio to OGG format with Opus codec
def convert_to_opus(audio_segment):
    with io.BytesIO() as byte_stream:
        audio_segment.export(byte_stream, format="ogg", codec="opus")
        byte_stream.seek(0)
        opus_audio_segment = AudioSegment.from_file(byte_stream, format="ogg")
    return opus_audio_segment

# Main function
def main():
    audio_data = record_audio()
    audio_segment = save_to_byte_stream(audio_data)

    if is_speech(audio_segment.raw_data):
        print("Voice detected")
        opus_audio_segment = convert_to_opus(audio_segment)
        # You can now use opus_audio_segment for further processing
    else:
        print("No voice detected")

if __name__ == "__main__":
    main()


Recording...
Voice detected


CouldntEncodeError: Encoding failed. ffmpeg/avlib returned error code: 3561836632

Command:['ffmpeg', '-y', '-f', 'wav', '-i', 'C:\\Users\\066226~1\\AppData\\Local\\Temp\\tmptx_8fzl3', '-acodec', 'opus', '-f', 'ogg', 'C:\\Users\\066226~1\\AppData\\Local\\Temp\\tmp6vy1nt5b']

Output from ffmpeg/avlib:

ffmpeg version N-112699-g5a2ca4bf7a-20231108 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 13.2.0 (crosstool-NG 1.25.0.232_c175b21)
  configuration: --prefix=/ffbuild/prefix --pkg-config-flags=--static --pkg-config=pkg-config --cross-prefix=x86_64-w64-mingw32- --arch=x86_64 --target-os=mingw32 --enable-gpl --enable-version3 --disable-debug --enable-shared --disable-static --disable-w32threads --enable-pthreads --enable-iconv --enable-libxml2 --enable-zlib --enable-libfreetype --enable-libfribidi --enable-gmp --enable-lzma --enable-fontconfig --enable-libharfbuzz --enable-libvorbis --enable-opencl --disable-libpulse --enable-libvmaf --disable-libxcb --disable-xlib --enab

In [14]:

import sounddevice as sd
import soundfile as sf
import io
from pydub import AudioSegment
from pydub.silence import detect_nonsilent
import time
import numpy as np

# Constants
RATE = 16000
CHANNELS = 1
CHUNK = 480
SILENCE_THRESHOLD = -50
REQUIRED_NONSILENT = 100
RECORD_SECONDS = 5

# Detect if there's voice activity
def is_speech(audio_data):
    audio_segment = AudioSegment(audio_data, sample_width=2, channels=CHANNELS, frame_rate=RATE)
    nonsilent_ranges = detect_nonsilent(audio_segment, min_silence_len=100, silence_thresh=SILENCE_THRESHOLD)
    if nonsilent_ranges and nonsilent_ranges[-1][1] - nonsilent_ranges[0][0] >= REQUIRED_NONSILENT:
        return True
    return False
# Record audio using sounddevice
def record_audio():
    print("Recording...")
    audio_data = sd.rec(frames=RECORD_SECONDS * RATE, samplerate=RATE, channels=CHANNELS, dtype='int16')
    sd.wait()
    return audio_data

# Save recorded audio to a temporary WAV file and read it as a byte stream
def save_to_byte_stream(audio_data):
    with io.BytesIO() as byte_stream:
        with sf.SoundFile(byte_stream, 'w', samplerate=RATE, channels=CHANNELS, format='WAV', subtype='PCM_16') as wav_file:
            wav_file.write(audio_data)
        byte_stream.seek(0)
        audio_segment = AudioSegment.from_file(byte_stream, format="wav")
    return audio_segment

# Main function
def main():
    audio_data = record_audio()
    audio_segment = save_to_byte_stream(audio_data)

    if is_speech(audio_segment.raw_data):
        print("Voice detected")
        # You can now use audio_segment for further processing
    else:
        print("No voice detected")

if __name__ == "__main__":
    main()


Recording...
Voice detected


In [20]:

import sounddevice as sd
import soundfile as sf
import io
from pydub import AudioSegment
from pydub.silence import detect_nonsilent
import time
import numpy as np

# Constants
RATE = 16000
CHANNELS = 1
CHUNK = 480
SILENCE_THRESHOLD = -50
REQUIRED_NONSILENT = 100
RECORD_SECONDS = 5

# Detect if there's voice activity
def is_speech(audio_data):
    audio_segment = AudioSegment(audio_data, sample_width=2, channels=CHANNELS, frame_rate=RATE)
    nonsilent_ranges = detect_nonsilent(audio_segment, min_silence_len=100, silence_thresh=SILENCE_THRESHOLD)

    if nonsilent_ranges and nonsilent_ranges[-1][1] - nonsilent_ranges[0][0] >= REQUIRED_NONSILENT:
        return True
    return False

# Record audio using sounddevice
def record_audio():
    print("Recording...")
    audio_data = sd.rec(frames=RECORD_SECONDS * RATE, samplerate=RATE, channels=CHANNELS, dtype='int16')
    sd.wait()
    return audio_data

# Save recorded audio to a temporary WAV file and read it as a byte stream
def save_to_byte_stream(audio_data):
    with io.BytesIO() as byte_stream:
        with sf.SoundFile(byte_stream, 'w', samplerate=RATE, channels=CHANNELS, format='WAV', subtype='PCM_16') as wav_file:
            wav_file.write(audio_data)
        byte_stream.seek(0)
        audio_segment = AudioSegment.from_file(byte_stream, format="wav")
    return audio_segment

# Play audio using sounddevice
def play_audio(audio_segment):
    print("Playing audio...")
    audio_data = np.array(audio_segment.get_array_of_samples(), dtype=np.int16)
    sd.play(audio_data, samplerate=RATE)
    sd.wait()

# Main function
def main():
    audio_data = record_audio()
    audio_segment = save_to_byte_stream(audio_data)

    if is_speech(audio_segment.raw_data):
        print("Voice detected")
        play_audio(audio_segment)
    else:
        print("No voice detected")

if __name__ == "__main__":
    main()


Recording...
Voice detected
Playing audio...


In [21]:
import sounddevice as sd
import soundfile as sf
import io
from pydub import AudioSegment
from pydub.silence import detect_nonsilent
import time
import numpy as np

# Constants
RATE = 16000
CHANNELS = 1
CHUNK = 480
SILENCE_THRESHOLD = -50
REQUIRED_NONSILENT = 100
RECORD_SECONDS = 5

def is_speech(audio_data):
    audio_segment = AudioSegment(audio_data, sample_width=2, channels=CHANNELS, frame_rate=RATE)
    nonsilent_ranges = detect_nonsilent(audio_segment, min_silence_len=100, silence_thresh=SILENCE_THRESHOLD)

    if nonsilent_ranges and nonsilent_ranges[-1][1] - nonsilent_ranges[0][0] >= REQUIRED_NONSILENT:
        return True
    return False

def record_audio():
    print("Recording...")
    audio_data = sd.rec(frames=RECORD_SECONDS * RATE, samplerate=RATE, channels=CHANNELS, dtype='int16')
    sd.wait()
    return audio_data

def save_to_byte_stream(audio_data):
    with io.BytesIO() as byte_stream:
        with sf.SoundFile(byte_stream, 'w', samplerate=RATE, channels=CHANNELS, format='WAV', subtype='PCM_16') as wav_file:
            wav_file.write(audio_data)
        byte_stream.seek(0)
        audio_segment = AudioSegment.from_file(byte_stream, format="wav")
    return audio_segment

def play_audio(audio_segment):
    print("Playing audio...")
    audio_data = np.array(audio_segment.get_array_of_samples(), dtype=np.int16)
    sd.play(audio_data, samplerate=RATE)
    sd.wait()

def main():
    while True:
        audio_data = record_audio()
        audio_segment = save_to_byte_stream(audio_data)

        if is_speech(audio_segment.raw_data):
            print("Voice detected")
            play_audio(audio_segment)
        #else:
        #    print("No voice detected")
        time.sleep(0.5)  # Optional: Add a short delay between iterations

if __name__ == "__main__":
    main()

Recording...
Voice detected
Playing audio...
Recording...
Voice detected
Playing audio...
Recording...
Voice detected
Playing audio...
Recording...
Recording...
Recording...


KeyboardInterrupt: 

In [31]:
import sounddevice as sd
import soundfile as sf
import io
from pydub import AudioSegment
from pydub.silence import detect_nonsilent
import time
import numpy as np

# Constants
RATE = 16000
CHANNELS = 1
CHUNK = 480
SILENCE_THRESHOLD = -50
REQUIRED_NONSILENT = 100
RECORD_SECONDS = 0.7  # Record in smaller chunks

def is_speech(audio_data):
    audio_segment = AudioSegment(audio_data, sample_width=2, channels=CHANNELS, frame_rate=RATE)
    nonsilent_ranges = detect_nonsilent(audio_segment, min_silence_len=100, silence_thresh=SILENCE_THRESHOLD)

    if nonsilent_ranges and nonsilent_ranges[-1][1] - nonsilent_ranges[0][0] >= REQUIRED_NONSILENT:
        return True
    return False

def record_audio():
    audio_data = sd.rec(frames=int(RECORD_SECONDS * RATE), samplerate=RATE, channels=CHANNELS, dtype='int16')
    sd.wait()
    return audio_data

def save_to_byte_stream(audio_data):
    with io.BytesIO() as byte_stream:
        with sf.SoundFile(byte_stream, 'w', samplerate=RATE, channels=CHANNELS, format='WAV', subtype='PCM_16') as wav_file:
            wav_file.write(audio_data)
        byte_stream.seek(0)
        audio_segment = AudioSegment.from_file(byte_stream, format="wav")
    return audio_segment

def play_audio(audio_segment):
    print("Playing audio...")
    audio_data = np.array(audio_segment.get_array_of_samples(), dtype=np.int16)
    sd.play(audio_data, samplerate=RATE)
    sd.wait()

def main():
    while True:
        recorded_audio = AudioSegment.empty()
        recording = False

        while True:
            audio_data = record_audio()
            audio_segment = save_to_byte_stream(audio_data)

            if is_speech(audio_segment.raw_data):
                if not recording:
                    print("Voice detected")
                    recording = True
                recorded_audio += audio_segment
            else:
                if recording:
                    print("Silence detected")
                    break

        if len(recorded_audio) > 0:
            play_audio(recorded_audio)
        #else:
        #    print("No voice detected")

        time.sleep(0.5)  # Optional: Add a short delay between iterations

if __name__ == "__main__":
    main()

Voice detected
Silence detected
Playing audio...
Voice detected
Silence detected
Playing audio...
Voice detected
Silence detected
Playing audio...
Voice detected
Silence detected
Playing audio...
Voice detected
Silence detected
Playing audio...
Voice detected
Silence detected
Playing audio...
Voice detected
Silence detected
Playing audio...
Voice detected
Silence detected
Playing audio...
Voice detected
Silence detected
Playing audio...
Voice detected
Silence detected
Playing audio...
Voice detected
Silence detected
Playing audio...
Voice detected
Silence detected
Playing audio...


KeyboardInterrupt: 

Voice detected
Silence detected
Playing audio...
Voice detected
Silence detected
Playing audio...
Voice detected
Silence detected
Playing audio...
Voice detected
Silence detected
Playing audio...
Voice detected
Silence detected
Playing audio...
Voice detected
Silence detected
Playing audio...
Voice detected
Silence detected
Playing audio...
Voice detected
Silence detected
Playing audio...
Voice detected
Silence detected
Playing audio...
Voice detected
Silence detected
Playing audio...
Voice detected
Silence detected
Playing audio...


KeyboardInterrupt: 

The issue of the last words or ending sounds being dropped during audio playback could be due to a combination of factors, including the way audio is recorded in smaller chunks, the silence detection method, and the specific recording setup. Here are some possible explanations and suggestions to improve the audio playback:

1. Incomplete recording of the last chunk: The last chunk of audio might not contain the complete ending sound or word, causing it to be cut off during playback. You can try increasing the `RECORD_SECONDS` value to record longer chunks, which might help capture the complete ending sound.

2. Silence detection threshold: The `SILENCE_THRESHOLD` and `REQUIRED_NONSILENT` values used in the `is_speech` function might be too strict, causing the last part of the speech to be considered as silence. You can try adjusting these values to better suit your specific recording environment and setup.

3. Insufficient padding after detecting silence: The code stops recording as soon as silence is detected, which might cause the last sound to be cut off. To address this, you can add a short padding time after detecting silence, as demonstrated in the previous answer.

In [ ]:
Here is the full code of the modified solution with increased `RECORD_SECONDS` and added `SILENCE_PADDING_TIME`:


This modified solution records audio in longer chunks of 1 second and adds a 0.5-second padding time after detecting silence to ensure the entire speech, including the ending sounds, is captured. You can further fine-tune the parameters based on your specific recording environment and setup.

In [34]:
import sounddevice as sd
import soundfile as sf
import io
from pydub import AudioSegment
from pydub.silence import detect_nonsilent
import time
import numpy as np

# Constants
RATE = 16000
CHANNELS = 1
CHUNK = 480
SILENCE_THRESHOLD = -50
REQUIRED_NONSILENT = 100
RECORD_SECONDS = 1.0  # Record in longer chunks
SILENCE_PADDING_TIME = 0.5  # Time to keep recording after silence is detected

def is_speech(audio_data):
    audio_segment = AudioSegment(audio_data, sample_width=2, channels=CHANNELS, frame_rate=RATE)
    nonsilent_ranges = detect_nonsilent(audio_segment, min_silence_len=100, silence_thresh=SILENCE_THRESHOLD)

    if nonsilent_ranges and nonsilent_ranges[-1][1] - nonsilent_ranges[0][0] >= REQUIRED_NONSILENT:
        return True
    return False

def record_audio():
    audio_data = sd.rec(frames=int(RECORD_SECONDS * RATE), samplerate=RATE, channels=CHANNELS, dtype='int16')
    sd.wait()
    return audio_data

def save_to_byte_stream(audio_data):
    with io.BytesIO() as byte_stream:
        with sf.SoundFile(byte_stream, 'w', samplerate=RATE, channels=CHANNELS, format='WAV', subtype='PCM_16') as wav_file:
            wav_file.write(audio_data)
        byte_stream.seek(0)
        audio_segment = AudioSegment.from_file(byte_stream, format="wav")
    return audio_segment

def play_audio(audio_segment):
    print("Playing audio...")
    audio_data = np.array(audio_segment.get_array_of_samples(), dtype=np.int16)
    sd.play(audio_data, samplerate=RATE)
    sd.wait()

def main():
    while True:
        recorded_audio = AudioSegment.empty()
        recording = False
        silence_start_time = None

        while True:
            audio_data = record_audio()
            audio_segment = save_to_byte_stream(audio_data)

            if is_speech(audio_segment.raw_data):
                if not recording:
                    print("Voice detected")
                    recording = True
                recorded_audio += audio_segment
                silence_start_time = None
            else:
                if recording:
                    if silence_start_time is None:
                        silence_start_time = time.time()
                    elif time.time() - silence_start_time >= SILENCE_PADDING_TIME:
                        print("Silence detected")
                        break

        if len(recorded_audio) > 0:
            play_audio(recorded_audio)
        else:
            print("No voice detected")

        #time.sleep(0.5)  # Optional: Add a short delay between iterations

if __name__ == "__main__":
    main()

Voice detected
Silence detected
Playing audio...
Voice detected
Silence detected
Playing audio...
Voice detected
Silence detected
Playing audio...
Voice detected
Silence detected
Playing audio...
Voice detected
Silence detected
Playing audio...
Voice detected
Silence detected
Playing audio...


KeyboardInterrupt: 

Voice detected
Silence detected
Playing audio...
Voice detected
Silence detected
Playing audio...
Voice detected
Silence detected
Playing audio...


KeyboardInterrupt: 